In [2]:
import os
import os.path as osp
import json
import matplotlib.pyplot as plt
import  numpy as np
from glob import glob
import matplotlib.animation as animation
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
import pandas as pd
import cv2


In [3]:
root_folder = '/videos/mpi_data/2Itzik/MPIIGroupInteraction'
all_pose_files = glob(osp.join(root_folder,'openpose_train', '*.json'))
all_video_files = glob(osp.join(root_folder,'clips_train', '*.mp4'))

In [99]:
import json
import pandas as pd
# pose_path = all_pose_files[0]
pose_path = osp.join(root_folder,'openpose_train', '21185-video1.json')
pose_path2 = osp.join(root_folder, 'openface_train', '21185-video1.csv')
video_path = osp.join(root_folder, 'clips_train', '21185-video1.mp4')
with open(pose_path, 'r') as f:
    pose_data = json.load(f)

df_face = pd.read_csv(pose_path2)


Index(['frame', 'face_id', 'timestamp', 'confidence', 'success', 'gaze_0_x',
       'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y',
       ...
       'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c',
       'AU26_c', 'AU28_c', 'AU45_c'],
      dtype='object', length=714)

In [104]:
df_face
a=1

In [100]:
[print(c) for c in df_face.columns]
print()

frame
face_id
timestamp
confidence
success
gaze_0_x
gaze_0_y
gaze_0_z
gaze_1_x
gaze_1_y
gaze_1_z
gaze_angle_x
gaze_angle_y
eye_lmk_x_0
eye_lmk_x_1
eye_lmk_x_2
eye_lmk_x_3
eye_lmk_x_4
eye_lmk_x_5
eye_lmk_x_6
eye_lmk_x_7
eye_lmk_x_8
eye_lmk_x_9
eye_lmk_x_10
eye_lmk_x_11
eye_lmk_x_12
eye_lmk_x_13
eye_lmk_x_14
eye_lmk_x_15
eye_lmk_x_16
eye_lmk_x_17
eye_lmk_x_18
eye_lmk_x_19
eye_lmk_x_20
eye_lmk_x_21
eye_lmk_x_22
eye_lmk_x_23
eye_lmk_x_24
eye_lmk_x_25
eye_lmk_x_26
eye_lmk_x_27
eye_lmk_x_28
eye_lmk_x_29
eye_lmk_x_30
eye_lmk_x_31
eye_lmk_x_32
eye_lmk_x_33
eye_lmk_x_34
eye_lmk_x_35
eye_lmk_x_36
eye_lmk_x_37
eye_lmk_x_38
eye_lmk_x_39
eye_lmk_x_40
eye_lmk_x_41
eye_lmk_x_42
eye_lmk_x_43
eye_lmk_x_44
eye_lmk_x_45
eye_lmk_x_46
eye_lmk_x_47
eye_lmk_x_48
eye_lmk_x_49
eye_lmk_x_50
eye_lmk_x_51
eye_lmk_x_52
eye_lmk_x_53
eye_lmk_x_54
eye_lmk_x_55
eye_lmk_y_0
eye_lmk_y_1
eye_lmk_y_2
eye_lmk_y_3
eye_lmk_y_4
eye_lmk_y_5
eye_lmk_y_6
eye_lmk_y_7
eye_lmk_y_8
eye_lmk_y_9
eye_lmk_y_10
eye_lmk_y_11
eye_lmk_y_12


In [56]:
print(pose_path)
print(video_path)

/videos/mpi_data/2Itzik/MPIIGroupInteraction/openpose_train/35975-video.json
/videos/mpi_data/2Itzik/MPIIGroupInteraction/clips_train/24628-video.mp4


In [81]:
pose_data['0']['body'][0]

[{'0': [[552, 208], 0.9559653997421265],
  '1': [[581, 281], 0.9455381035804749],
  '2': [[497, 275], 0.897963285446167],
  '3': [[460, 441], 0.9622132778167725],
  '4': [[528, 512], 1.0119996070861816],
  '5': [[668, 287], 0.9323213696479797],
  '6': [[679, 448], 0.9613420963287354],
  '7': [[623, 512], 0.9603403210639954],
  '8': [[512, 495], 0.8450831770896912],
  '9': [[427, 567], 0.9443049430847168],
  '10': [[425, 819], 0.7593596577644348],
  '11': [[629, 499], 0.8516924977302551],
  '12': [[712, 553], 0.9593846201896667],
  '13': [[726, 807], 0.8370798826217651],
  '14': [[536, 190], 1.0038281679153442],
  '15': [[578, 190], 0.9766519069671631],
  '16': [[506, 180], 0.37544965744018555],
  '17': [[620, 184], 0.9700128436088562]},
 32.68116801997085]

In [49]:
NUM_POSE_HAND = 21
NUM_POSE_BODY = 18
hand_th = 0.8
body_th = 0.8
WIDTH, HEIGHT = int(1224), int(1624)


In [17]:
len(pose_data.keys())

64

In [84]:
def blur_and_normalize_frames(frames, sigma=1):
    blurred_frames = np.zeros_like(frames, dtype=float)
    n_frames = frames.shape[0]
    # Iterate over each frame
    for i in range(n_frames):
        frame = frames[i, :,:]
        # Apply Gaussian blur
        blurred_frame = gaussian_filter(frame, sigma=sigma)
        # Normalize frame by dividing by maximum value
        max_value = np.max(blurred_frame)
        blurred_frames[i, :,:] = blurred_frame / max_value

    return blurred_frames

def draw_pose(pose_data):
    num_frames = len(pose_data.keys())
    num_pose_body = NUM_POSE_BODY
    num_pose_hand = NUM_POSE_HAND
    width, height = WIDTH, HEIGHT
    poses = []
    canvas = np.zeros((num_frames, width, height))
    for frame_number_txt, frame_data in tqdm(pose_data.items(), total=64):
        poses_frame = []
        frame_number = int(frame_number_txt)
        for body_part, joint_data in frame_data.items():
            if body_part == 'body':
                for joint_number, (coords, confidence) in joint_data[0][0].items():
                    x, y = int(coords[0]), int(coords[1])
                    if confidence>body_th:
                        canvas[int(frame_number), x, y] = 1
                        poses_frame.append((x,y))
                        

            elif body_part == 'hand':
                for hand in joint_data:
                    for joint_number, coords in hand.items():
                        x, y = int(coords[0]), int(coords[1])
                        canvas[int(frame_number), x, y] = 1
                        poses_frame.append((x,y))
        poses.append(poses_frame)
    return canvas, poses



In [85]:
frames, poses = draw_pose(pose_data)


100%|██████████| 64/64 [00:00<00:00, 2905.99it/s]


In [86]:
# frames = blur_and_normalize_frames(frames, sigma=1)

In [87]:
# n_frames = frames.shape[0]
# for i in range(n_frames):
#     frame = frames[i, :, :]
#     # Scale the frame to 0-255 range and convert to uint8
#     frame_scaled = (255 * frame).astype(np.uint8)
    
#     # Display the frame
#     cv2.imshow('Frame', frame_scaled)
    
#     # Wait for key press (30 milliseconds per frame)
#     if cv2.waitKey(30) & 0xFF == ord('q'):
#         break

# # Release resources and close OpenCV window
# cv2.destroyAllWindows()

In [88]:
frames.shape

(64, 1224, 1624)

In [89]:
# display_frames(frames)


In [90]:
# def create_animation(frames):
#     nframe, width, height = frames.shape
#     fig = plt.figure(figsize=(width, height), dpi=30)
#     plt.axis('off')  
#     ims = []
#     for frame_ind in range(nframe):
        
#         im = plt.imshow(frames[frame_ind, :, :], animated=True, cmap='gray')
        
#         ims.append([im])
#     ani = animation.ArtistAnimation(fig, ims, interval=20, blit=True)
    
#     # if not save_path is None:
#     #     ani.save(save_path, writer='ffmpeg', fps=30) 
#     plt.close()

#     return ani.to_jshtml()

# Example usage:
# ani = create_animation(frames[:5])
# ani.save('animation.mp4', writer='ffmpeg', dpi=100)

In [91]:
# canvas.shape[0]

In [92]:

import ipywidgets as widgets
from IPython.display import display

# Function to display the video player
def play_video(video_path, pose_frames):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Create a display widget
    out = widgets.Output()
    display(out)
    
    # Function to display each frame of the video
    def display_frame(frame):
        out.clear_output(wait=True)
        with out:
            display(frame)
    
    # Display each frame of the video
    f = 0
    while cap.isOpened():
        if f > 10:
            break
        else:
            f += 1
            
        ret, frame = cap.read()
        if not ret:
            break
        # frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        frame_rgb = frame
        
        w,h, c = frame.shape
        # print(w,h)


             # Draw pose keypoints on the frame
        if len(pose_frames) > 0:
            keypoints = pose_frames[f]  # Get keypoints for the current frame
            for point in keypoints:
                x, y = int(point[0]), int(point[1])  # Extract x, y coordinates
                cv2.circle(frame_rgb, (x, y), 5, (0, 255, 0), -1)  # Draw a circle at each keypoint
                print(f'x:{x} y:{y}')
        
        frame_rgb = cv2.resize(frame_rgb, (w//2, h//2))  # Resize the frame to fit the display widget
        display_frame(widgets.Image(value=cv2.imencode('.jpg', frame_rgb)[1].tobytes()))

        # Optional: Add a small delay to control the playback speed
        # Adjust the value (in milliseconds) as needed
        
        key = cv2.waitKey() & 0xFF  # Masking to get the last 8 bits for ASCII value
        
        # If the 'q' key is pressed, break the loop and stop the video playback
        if key == ord('q'):
            break
    
    # Release the video capture object and close any OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

In [93]:
play_video(video_path, pose_frames=poses)

Output()

x:550 y:207
x:582 y:280
x:500 y:275
x:461 y:437
x:526 y:508
x:668 y:286
x:681 y:442
x:630 y:504
x:513 y:494
x:427 y:571
x:631 y:498
x:712 y:557
x:727 y:806
x:535 y:190
x:576 y:189
x:620 y:182
x:519 y:513
x:538 y:508
x:583 y:504
x:571 y:496
x:559 y:489
x:544 y:521
x:547 y:533
x:550 y:541
x:555 y:552
x:534 y:531
x:543 y:552
x:549 y:570
x:553 y:582
x:532 y:540
x:542 y:558
x:546 y:570
x:552 y:582
x:530 y:548
x:535 y:557
x:542 y:565
x:636 y:531
x:515 y:509
x:536 y:508
x:582 y:503
x:571 y:497
x:559 y:490
x:544 y:522
x:548 y:535
x:551 y:542
x:554 y:553
x:533 y:530
x:545 y:555
x:550 y:571
x:553 y:582
x:534 y:545
x:543 y:561
x:546 y:571
x:552 y:582
x:530 y:551
x:538 y:561
x:541 y:566
x:543 y:568


x:551 y:208
x:582 y:282
x:499 y:279
x:461 y:432
x:518 y:502
x:668 y:286
x:683 y:440
x:631 y:503
x:514 y:496
x:427 y:572
x:631 y:500
x:712 y:555
x:725 y:804
x:535 y:190
x:578 y:190
x:620 y:183
x:528 y:512
x:535 y:503
x:550 y:500
x:563 y:500
x:572 y:495
x:538 y:518
x:541 y:528
x:544 y:536
x:547 y:545
x:527 y:526
x:538 y:550
x:543 y:563
x:560 y:588
x:529 y:539
x:539 y:557
x:550 y:577
x:561 y:590
x:528 y:545
x:548 y:568
x:555 y:581
x:548 y:573
x:513 y:508
x:532 y:503
x:550 y:500
x:562 y:500
x:572 y:496
x:537 y:517
x:540 y:528
x:544 y:536
x:547 y:546
x:528 y:527
x:537 y:550
x:544 y:563
x:560 y:589
x:530 y:542
x:539 y:559
x:549 y:577
x:561 y:590
x:530 y:550
x:547 y:568
x:555 y:580
x:561 y:589


x:551 y:208
x:579 y:283
x:495 y:283
x:458 y:430
x:515 y:497
x:666 y:284
x:687 y:433
x:635 y:499
x:510 y:496
x:426 y:575
x:628 y:497
x:714 y:557
x:725 y:805
x:535 y:190
x:578 y:190
x:619 y:183
x:630 y:511
x:622 y:532
x:646 y:523
x:645 y:522
x:617 y:549
x:602 y:566
x:589 y:578
x:580 y:587
x:613 y:540
x:598 y:559
x:586 y:568
x:575 y:578
x:608 y:529
x:595 y:542
x:584 y:550
x:574 y:560
x:603 y:516
x:590 y:518
x:581 y:520
x:570 y:524
x:508 y:500
x:525 y:496
x:540 y:492
x:550 y:490
x:566 y:484
x:534 y:510
x:537 y:521
x:538 y:531
x:540 y:538
x:528 y:522
x:533 y:539
x:539 y:556
x:545 y:563
x:528 y:532
x:534 y:550
x:544 y:566
x:556 y:579
x:524 y:537
x:544 y:560
x:549 y:569
x:558 y:581


x:551 y:208
x:580 y:279
x:496 y:279
x:459 y:428
x:519 y:496
x:665 y:282
x:695 y:430
x:638 y:497
x:514 y:494
x:427 y:572
x:629 y:496
x:713 y:559
x:725 y:804
x:535 y:190
x:578 y:189
x:620 y:182
x:629 y:505
x:640 y:515
x:544 y:482
x:556 y:477
x:561 y:469
x:621 y:542
x:602 y:565
x:593 y:573
x:584 y:581
x:615 y:533
x:598 y:554
x:585 y:563
x:576 y:571
x:611 y:522
x:596 y:535
x:583 y:543
x:573 y:553
x:607 y:510
x:593 y:511
x:583 y:513
x:573 y:517
x:506 y:493
x:522 y:487
x:542 y:482
x:556 y:478
x:561 y:469
x:534 y:501
x:535 y:513
x:535 y:522
x:541 y:555
x:527 y:512
x:532 y:529
x:537 y:546
x:541 y:556
x:525 y:521
x:533 y:540
x:545 y:559
x:555 y:571
x:521 y:527
x:538 y:538
x:547 y:559
x:555 y:571


...

x:548 y:203
x:583 y:280
x:501 y:270
x:467 y:406
x:665 y:288
x:704 y:439
x:657 y:482
x:522 y:492
x:428 y:569
x:630 y:499
x:713 y:560
x:726 y:803
x:533 y:187
x:573 y:186
x:618 y:178
x:696 y:440
x:472 y:416
x:508 y:474
x:515 y:462
x:518 y:448
x:525 y:437
x:477 y:464
x:473 y:477
x:480 y:483
x:484 y:485
x:465 y:465
x:470 y:475
x:478 y:482
x:485 y:485
x:459 y:464
x:466 y:472
x:469 y:472
x:500 y:541
x:454 y:453
x:461 y:467
x:515 y:539
x:522 y:547


x:549 y:203
x:585 y:274
x:504 y:267
x:462 y:452
x:666 y:281
x:694 y:424
x:677 y:485
x:527 y:496
x:433 y:561
x:632 y:498
x:712 y:553
x:725 y:803
x:533 y:187
x:574 y:185
x:618 y:175
x:664 y:477
x:687 y:482
x:694 y:482
x:702 y:478
x:700 y:518
x:697 y:509
x:661 y:507
x:688 y:530
x:678 y:538
x:670 y:548
x:666 y:554


x:548 y:202
x:582 y:275
x:498 y:275
x:665 y:277
x:698 y:468
x:439 y:553
x:635 y:497
x:710 y:554
x:724 y:801
x:532 y:186
x:573 y:184
x:618 y:176
x:734 y:476
x:734 y:511


x:549 y:201
x:582 y:273
x:497 y:271
x:474 y:404
x:665 y:276
x:710 y:457
x:531 y:480
x:436 y:557
x:640 y:485
x:716 y:562
x:724 y:800
x:533 y:185
x:574 y:183
x:617 y:176
x:702 y:503
x:719 y:519
x:725 y:531
x:721 y:512
x:739 y:519
x:722 y:496
x:734 y:509
x:740 y:517
x:747 y:527
x:721 y:487
x:735 y:494
x:745 y:499
x:756 y:506
x:719 y:475
x:731 y:474
x:739 y:472
x:749 y:471
x:447 y:482
x:448 y:464
x:442 y:449
x:436 y:437
x:431 y:426
x:445 y:441
x:435 y:431
x:431 y:424
x:406 y:486
x:428 y:424
x:410 y:471
x:386 y:473
x:373 y:476
x:361 y:479


x:549 y:199
x:582 y:275
x:499 y:273
x:471 y:398
x:442 y:451
x:663 y:277
x:529 y:479
x:433 y:564
x:638 y:486
x:715 y:565
x:724 y:800
x:534 y:183
x:575 y:181
x:617 y:175
x:713 y:503
x:729 y:518
x:739 y:532
x:743 y:545
x:735 y:508
x:746 y:514
x:759 y:501
x:767 y:508
x:734 y:497
x:750 y:498
x:751 y:519
x:757 y:527
x:733 y:488
x:748 y:494
x:757 y:500
x:757 y:528
x:729 y:476
x:741 y:474
x:749 y:472
x:758 y:471
x:436 y:475
x:434 y:461
x:430 y:448
x:424 y:438
x:421 y:427
x:405 y:510
x:399 y:522
x:395 y:531
x:392 y:541
x:396 y:491
x:376 y:508
x:370 y:519
x:361 y:526
x:372 y:493
x:359 y:501
x:0 y:506
x:372 y:473
x:359 y:475
x:0 y:476
